In [1]:
import indicoio
import os
import json
import pandas as pd

In [2]:
# Turn JSON lines into JSON array
def json_line_to_array(filename):
    with open(filename) as f:
        lines = f.readlines()
        lines = map(lambda line: line.rstrip(), lines) #remove \n
        json_str = '[' + ','.join(lines) + ']'
        return json_str

In [53]:
# Read in data from sample of /r/politics users
# reddit_pos = pd.read_json('data/politicos/json/multi.json')
# reddit_neg = pd.read_json('data/politicos/json/single.json')
reddit_multi = pd.read_json(json_line_to_array('data/politicos/json/balanced_multi.json'))
reddit_single = pd.read_json(json_line_to_array('data/politicos/json/balanced_single.json'))

In [54]:
#first_post = pd.concat([reddit_pos, reddit_neg], axis=0)
#first_post.set_index('author', inplace=True)

In [55]:
indicoio.config.api_key = os.environ['INDICO_API_KEY']

In [56]:
def get_sentiments(comments, step_size):
    '''
        Call sentiment API in batches
        
        Ex: Call API:    ['libertarian comment 1', 'green comment']
            API returns: [{u'Libertarian': 0.3500982762633169, u'Green': 0.14290592846839006, u'Liberal': 0.3230786607985243, u'Conservative': 0.18391713446976865},
                          {u'Libertarian': 0.26326212703421853, u'Green': 0.3227391644798419, u'Liberal': 0.3722932170134808, u'Conservative': 0.041705491472458736}]
    '''
    sentiments = []
    for start in xrange(0, len(comments), step_size):
        step = step_size if start+step_size<len(comments) else len(comments)-start
        comments_batch = list(comments[start:start+step]['first_post_body'])
        pol_sentiment_batch = indicoio.political(comments_batch)
        for pol_dict in pol_sentiment_batch:
            party = max(pol_dict, key=pol_dict.get)
            sentiments.append(party)
    return sentiments

In [57]:
# Get political sentiment of multi-posters
pol_parties_multi = get_sentiments(reddit_multi, 1000)
with open('data/politicos/json/balanced_multi_pol.csv', 'w') as f:
    f.write('\n'.join(pol_parties_multi))

In [58]:
# Get political sentiment of single posters
pol_parties_single = get_sentiments(reddit_single, 1000)
with open('data/politicos/json/balanced_single_pol.csv', 'w') as f:
    f.write('\n'.join(pol_parties_single))

In [52]:
# Read in data as follows in model-building script (merge with comments data)
# pd.read_csv('data/politicos/json/balanced_multi_pol.csv', header=None, names=['pol_party'])

,pol_party
0,Libertarian
1,Conservative
2,Liberal
3,Libertarian
4,Liberal


In [34]:
# Test
# test_statements = ['big government is the worst', 
#                    'education is important for our future', 
#                    'we need progressive policies for working families',
#                    'we need to reduce government waste',
#                    'we need renewable energy to stop climate change']
# test_df = pd.DataFrame(test_statements)
# test_df.columns = ['first_post_body']
# sentiments = get_sentiments(test_df, 3)
# test_df['pol_party'] = sentiments
# test_df

,first_post_body,sentiments
0,big government is the worst,Libertarian
1,education is important for our future,Green
2,we need progressive policies for working families,Liberal
3,we need to reduce government waste,Libertarian
4,we need renewable energy to stop climate change,Green


In [19]:
# indicoio.political(test_statements)

[{u'Conservative': 0.15061259920567033,
  u'Green': 0.14817878436127507,
  u'Liberal': 0.11490891839615226,
  u'Libertarian': 0.5862996980369023},
 {u'Conservative': 0.11938713690826618,
  u'Green': 0.510927010838791,
  u'Liberal': 0.22709480964599627,
  u'Libertarian': 0.14259104260694658},
 {u'Conservative': 0.093010169684295,
  u'Green': 0.0435243999854284,
  u'Liberal': 0.8246997267928402,
  u'Libertarian': 0.03876570353743644},
 {u'Conservative': 0.15833398427551645,
  u'Green': 0.34813628450062045,
  u'Liberal': 0.07130184509205965,
  u'Libertarian': 0.4222278861318033},
 {u'Conservative': 0.0034786444891636483,
  u'Green': 0.9847222555497523,
  u'Liberal': 0.010172260986774161,
  u'Libertarian': 0.0016268389743097102}]